In [27]:
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
import numpy as np
from queue import PriorityQueue
from collections import deque

In [28]:
class NPuzzle:
    """
    I started by creating a NPuzzle function. This class allow me to represent the problem.
    Instances:
        This class contains as instance the dimention of the puzzle (entered by the user) and 3 states of the puzzle.
        The first one is the goal state with all of the piece of the puzzle in the right order. 
        The second one is the the current state named simply 'state' which contain the current order of the pieces of the puzzle.
        The last one is the initial state which is equal to the current state during the intialisation.
    """
    def __init__(self, PUZZLE_DIM=5, RANDOMIZE_STEPS = 100_000):
        self.action=namedtuple('Action', ['pos1', 'pos2'])  #First pos is the original position of 0 and second pos is where we want to put it
        self.PUZZLE_DIM=PUZZLE_DIM
        self.state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
        self.goal_state=self.state.copy()
        self.actions = list()
        if RANDOMIZE_STEPS>0:
            self.available_actions()
            for i in range(RANDOMIZE_STEPS):
                self.do_action(choice(self.actions))
        self.initial_state=self.state
        
    def available_actions(self): # This function detect where the 0 is in the puzzle and use this information to deduce where we can move next
        x, y = [int(_[0]) for _ in np.where(self.state == 0)]
        self.actions = list()
        if x > 0:
            self.actions.append(self.action((x, y), (x - 1, y)))
        if x < self.PUZZLE_DIM - 1:
            self.actions.append(self.action((x, y), (x + 1, y)))
        if y > 0:
            self.actions.append(self.action((x, y), (x, y - 1)))
        if y < self.PUZZLE_DIM - 1:
            self.actions.append(self.action((x, y), (x, y + 1)))

    def do_action(self,action): #Modify the current state by doing a specific action
        new_state = self.state
        new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
        self.state=new_state.copy()
        self.available_actions() #Once the puzzle modified the next possible moves are recalculated.
        
    def manhatan_distance(self,tile): #The manhatan distance is the sum of the x and y axes length between where a tile is and where it is supposed to be (current state VS final state)
        i=np.where(self.state==tile)
        j=np.where(self.goal_state==tile)
        row_i, col_i = i[0][0], i[1][0]
        row_j, col_j = j[0][0], j[1][0]
        return abs(row_i-row_j) + abs(col_i-col_j)
    
    def goal_test(self): #Check if the current state equal the goal state.
        return np.array_equal(self.state, self.goal_state)
    
    def successors(self): #Calculate every possible direct successor of the puzzle using the list of next possible actions
        successors = []
        for action in self.actions:
            new_puzzle = NPuzzle(PUZZLE_DIM=self.PUZZLE_DIM, RANDOMIZE_STEPS=0)
            new_puzzle.state = self.state.copy()
            new_puzzle.do_action(action)
            successors.append((new_puzzle, action, 1))  # Step cost is 1 for each move
        return successors

In [29]:
class Node:
    """
    The Node class allow me to keep track of the number of node created and evaluated.
    It associate a puzzle with his parent, the previous action used to arrive to this node and the cost from the original state.
    Creating a new puzzle for each node allow us to not modifie the current state of the original puzzle. We can then create a new node coming from any already created one and not only the current or intial node.
    """
    def __init__(self, puzzle, parent=None, action=None, cost=0):
        self.puzzle = puzzle
        self.parent = parent
        self.action = action
        self.cost = cost #The cost here is the minimal number of moves made found to obtain the puzzle (it is different from the final cost which is the number of created nodes)
        
    def __lt__(self,other): #This function make the program understand that the value of the node is the cost (when comparing nodes)
        return self.cost<other.cost

In [30]:
"""
Here I have determine 2 possible heuristics and a third one combining the two.
The idea is to try different heuristics for the A* search.
"""
def total_manhattan_distance(puzzle): #This is the sum of manhatan distance for each tile.
    return sum(puzzle.manhatan_distance(tile) for tile in range(1, puzzle.PUZZLE_DIM**2))
    
def misplaced_tiles(puzzle): #Number of tiles not at the right place
    count = 0
    for i in range(puzzle.PUZZLE_DIM):
        for j in range(puzzle.PUZZLE_DIM):
            if puzzle.state[i,j]!=0 and puzzle.state[i,j]!=puzzle.goal_state[i,j]:
                count+=1
    return count
    
def combined_heuristics(puzzle): #mixt of the two precedent heuritics
    return (total_manhattan_distance(puzzle) + misplaced_tiles(puzzle))*0.75 
#I multiplied the output by 0.75 because here the output will be bigger compared to the other heuritics (because it is the sum of two heuristics).
#When I tried to use it with A* the results were not really good and I think it was because the heuristic value was to big compare to the cost and so the cost was overlooked and the results bad.

In [31]:
def a_star_search(puzzle, heuristic):
    """
    Perform A* search on a given puzzle using a specified heuristic.
    Inputs:
        puzzle: The initial puzzle state.
        heuristic: The heuristic function to evaluate the state.
    Outputs:
        list of actions
        list of states
        Number of states generated
    """
    num_generated =0
    frontier=PriorityQueue()  # I use the PriorityQueue to be able to select the node with the less cost
    reached=dict()  #Keep track of the created state of puzzle to avoid creating multiple nodes

    node = Node(puzzle, None, None, 0) #I create the intial node, it has no parent, no privious action and a cost of 0
    frontier.put((heuristic(puzzle), node))  #It is added to the queue
    reached[tuple(puzzle.state.flatten())]=node #Use of a tuple to store the already visited state because it is faster to compare the different puzzles in this format. (I looked for solutions to make my code quicker)

    while not frontier.empty():
        _, node = frontier.get()  #Get the node with the lowest cost

        # We check if we reached the goal
        if node.puzzle.goal_test():
            actions,states=solution(node)
            return actions, states, num_generated

        #We expand node
        for successor, action, step_cost in node.puzzle.successors():
            num_generated+=1 #We considere as evaluated every child node we add to the tree. Even if we already reached this state we are reavaluating the cost to see if it is cheaper. So we are technically evaluating a new action.
            path_cost=node.cost+step_cost  #Calculating the path cost
            state_tuple=tuple(successor.state.flatten())

            if state_tuple not in reached or path_cost<reached[state_tuple].cost: #The node is only interresting to us if we never reached this state or only with more steps before.
                child_node=Node(successor, node, action, path_cost)
                reached[state_tuple]=child_node
                frontier.put((path_cost + 5*heuristic(successor), child_node)) 
                #Here i added a 5 to the heuristic so that it could be taking more into account.
                #At first I tried without adding any coefficient and the program was very slow to converge for small dimention and never converge for big dimentions.
                #I think it may be because the path cost had too much influence and so the program was always restarting from a node close to the original state to minimize the total value (path cost + heuristic)
                #By giving more importance to heuristic the program will look ahead and put more effort into minimize the heuritic and consequently getting closer to the goal.
    
    return None,None, num_generated  #Return None if no solution is found


In [32]:
def solution(node):
    #Allow to get all the actions and states to go from intial state to goal state using the path determined by the algorithm.
    actions = []
    states = []
    while node.parent is not None:
        actions.append(node.action)
        states.append(node.puzzle.state.copy())
        node = node.parent
    states.append(node.puzzle.state.copy()) 
    return actions[::-1], states[::-1] 

In [33]:
puzzle = NPuzzle(PUZZLE_DIM=5, RANDOMIZE_STEPS=100_000)

In [34]:
puzzle.initial_state

array([[ 8,  5, 16,  4, 10],
       [11,  9, 12, 21,  6],
       [22, 14,  0,  2, 18],
       [13,  1, 15,  7, 17],
       [20,  3, 19, 23, 24]])

In [37]:
# Solve using total_manhattan_distance heuristic
actions, states, num_generated = a_star_search(puzzle, heuristic=total_manhattan_distance)

# Print results
print("Quality using manhattan distance:", len(actions))
print("Cost:", num_generated)
print("States leading to the goal:")
for state in states:
    print(state,"\n")

Quality using manhattan distance: 176
Cost: 43062
States leading to the goal:
[[ 8  5 16  4 10]
 [11  9 12 21  6]
 [22 14  0  2 18]
 [13  1 15  7 17]
 [20  3 19 23 24]] 

[[ 8  5 16  4 10]
 [11  9  0 21  6]
 [22 14 12  2 18]
 [13  1 15  7 17]
 [20  3 19 23 24]] 

[[ 8  5 16  4 10]
 [11  9 21  0  6]
 [22 14 12  2 18]
 [13  1 15  7 17]
 [20  3 19 23 24]] 

[[ 8  5 16  4 10]
 [11  9 21  2  6]
 [22 14 12  0 18]
 [13  1 15  7 17]
 [20  3 19 23 24]] 

[[ 8  5 16  4 10]
 [11  9 21  2  6]
 [22 14 12  7 18]
 [13  1 15  0 17]
 [20  3 19 23 24]] 

[[ 8  5 16  4 10]
 [11  9 21  2  6]
 [22 14 12  7 18]
 [13  1  0 15 17]
 [20  3 19 23 24]] 

[[ 8  5 16  4 10]
 [11  9 21  2  6]
 [22 14 12  7 18]
 [13  1 19 15 17]
 [20  3  0 23 24]] 

[[ 8  5 16  4 10]
 [11  9 21  2  6]
 [22 14 12  7 18]
 [13  1 19 15 17]
 [20  0  3 23 24]] 

[[ 8  5 16  4 10]
 [11  9 21  2  6]
 [22 14 12  7 18]
 [13  1 19 15 17]
 [ 0 20  3 23 24]] 

[[ 8  5 16  4 10]
 [11  9 21  2  6]
 [22 14 12  7 18]
 [ 0  1 19 15 17]
 [13 20  3 23

In [36]:
actions, states, num_generated = a_star_search(puzzle, heuristic=combined_heuristics)

# Print results
print("Quality Combined Heuritics:", len(actions))
print("Cost:", num_generated)
print("States leading to the goal:")
for state in states:
    print(state,"\n")

Quality Combined Heuritics: 224
Cost: 53795
States leading to the goal:
[[ 8  5 16  4 10]
 [11  9 12 21  6]
 [22 14  0  2 18]
 [13  1 15  7 17]
 [20  3 19 23 24]] 

[[ 8  5 16  4 10]
 [11  9  0 21  6]
 [22 14 12  2 18]
 [13  1 15  7 17]
 [20  3 19 23 24]] 

[[ 8  5 16  4 10]
 [11  9 21  0  6]
 [22 14 12  2 18]
 [13  1 15  7 17]
 [20  3 19 23 24]] 

[[ 8  5 16  4 10]
 [11  9 21  2  6]
 [22 14 12  0 18]
 [13  1 15  7 17]
 [20  3 19 23 24]] 

[[ 8  5 16  4 10]
 [11  9 21  2  6]
 [22 14 12  7 18]
 [13  1 15  0 17]
 [20  3 19 23 24]] 

[[ 8  5 16  4 10]
 [11  9 21  2  6]
 [22 14 12  7 18]
 [13  1  0 15 17]
 [20  3 19 23 24]] 

[[ 8  5 16  4 10]
 [11  9 21  2  6]
 [22 14 12  7 18]
 [13  1 19 15 17]
 [20  3  0 23 24]] 

[[ 8  5 16  4 10]
 [11  9 21  2  6]
 [22 14 12  7 18]
 [13  1 19 15 17]
 [20  3 23  0 24]] 

[[ 8  5 16  4 10]
 [11  9 21  2  6]
 [22 14 12  7 18]
 [13  1 19 15 17]
 [20  3 23 24  0]] 

[[ 8  5 16  4 10]
 [11  9 21  2  6]
 [22 14 12  7 18]
 [13  1 19 15  0]
 [20  3 23 24 17]] 

In [ ]:
#It does not converge
# Solve using misplaced_tiles heuristic
actions, states, num_generated = a_star_search(puzzle, heuristic=misplaced_tiles)

# Print results
print("Quality using Misplaced Tiles in:", len(actions))
print("Cost:", num_generated)
print("States leading to the goal:")
for state in states:
    print(state,"\n")
